In [1]:
import pandas as pd
import numpy as np

In [2]:
def logprior(b, sigma2):
    return -np.log(sigma2)

def logistic(x, b):
    #x[0]: t
    #x[1]: h
    #x[2]: w
    theta = b[0] + x[0]*b[1] + x[1]*b[2] + x[2]*b[3]
    return 0.5/(1+np.exp(theta)) + 1

def loglikelihood(b, sigma2, x, y):
    n = len(x)
    return -n*np.log(sigma2) - 0.5*((y-logistic(x, b))**2).sum()/sigma2

def logpost(b, sigma2, x, y):
    return logprior(b, sigma2) + loglikelihood(b, sigma2, x, y)

In [ ]:
# mcmc algorithm
def mcmc(b_init, sig2, x, y, opt1, opt2, N, burnin, be, sig2e):
    B = np.zeros((N,len(b_init)))
    Sig2 = np.zeros(N)
    
    b_prev = b_init
    sig2_prev = sig2
    count = 0
    r = np.random.random(N)
    
    for i in xrange(N):
        b_star = np.random.normal(b_prev,be)

        sigma2new = abs(np.random.normal(sigma2old, sig2e))
        
        p = logpost(b_star, sig2_star, x, y) - logpost(b_prev, sig2_prev, x, y)
        if np.log(r[i]) < p:
            b_prev = b_star
            sig2_prev = sig2_star
            count += 1
                           
        B[i] = b_prev
        Sig2[i] = sig2_prev
    print "The acceptance rate is "+ str(float(count)/N)+"."
    return np.vstack((B[burnin:],Sig2[burnin:]))